In [1]:
# import library
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster

In [2]:
# read data
school_data = pd.read_csv("./data/curated/school_location_cleaned_2023.csv")
train_data = pd.read_csv("./data/curated/train_station_cleaned.csv")
population_data = pd.read_csv("./data/curated/population_cleaned.csv")
rent_data = pd.read_csv("./data/curated/rent_cleaned.csv")

In [3]:
# Load the map file for post codes
shapefile_data = gpd.read_file("./data/raw/external/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
shapefile_data = shapefile_data.rename(columns={'SA2_2021': 'Postcode'})

In [4]:
# define function to extract location points
def extract_location_points(lat_list, lng_list, additional_data):
    points = []
    for lat, lng, data in zip(lat_list, lng_list, additional_data):
        points.append((lat, lng, data))
    return points

In [5]:
# process school data
school_points = extract_location_points(school_data["Y"], school_data["X"], school_data["School_Type"])

# process train data
train_points = extract_location_points(train_data["latitude"], train_data["longitude"], train_data["Stop_ID"])

# process rent data
rent_points = extract_location_points(rent_data["latitude"], rent_data["longitude"], rent_data["price_per_week"])

# process population data
postcode_mapping = pd.read_csv("./data/raw/external/australian_postcodes.csv")[["SA2_CODE_2021", "lat", "long"]]
postcode_dict = {row['SA2_CODE_2021']: (row['lat'], row['long']) for _, row in postcode_mapping.iterrows()}

population_points = [(postcode_dict[suburb][0], postcode_dict[suburb][1], pop) 
                     for suburb, pop in zip(population_data["SA2 code"], population_data["2023"]) 
                     if suburb in postcode_dict]

In [6]:
# define function to add marker clusters
def create_marker_cluster(map_object, name, icon, color, data_points):
    feature_group = folium.FeatureGroup(name=name, control=True)
    marker_cluster = MarkerCluster(name=name).add_to(feature_group)
    for lat, lng, info in data_points:
        folium.Marker(
            location=[lat, lng],
            icon=folium.Icon(icon=icon, color=color, icon_color='white', prefix="fa"),
            popup=str(info)
        ).add_to(marker_cluster)
    map_object.add_child(feature_group)

In [7]:
# create map
map_object = folium.Map(location=[-36, 145.3906], tiles="OpenStreetMap", zoom_start=7)

In [8]:
# add markers on map
create_marker_cluster(map_object, 'School_Layer', 'university', 'black', school_points)
create_marker_cluster(map_object, 'Train_Layer', 'train', 'green', train_points)
create_marker_cluster(map_object, 'Rent_Layer', 'home', 'red', rent_points)
create_marker_cluster(map_object, 'Population_Layer', 'child', 'blue', population_points)

In [9]:
# add layer control
folium.LayerControl().add_to(map_object)

In [10]:
# save map to html file
map_object.save('livable.html')